# An intro to quantum machine learning 🚀
(with one qubit!)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_circles
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


Since we will only use one qubit, we need to reduce the dimensionality of the data. For this, we will use Kernel PCA.

In [ ]:
n_samples = 100

# Create the dataset
X, y = make_circles(n_samples=n_samples, noise=0.1, random_state=42, factor=0.4)

# Preprocess the dataset
scaler = MinMaxScaler((-np.pi, np.pi))
X = scaler.fit_transform(X)

# Plot the original dataset
plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
plt.title("Original Dataset")

plt.show()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

## Variational Quantum Classifier

In [ ]:
from qiskit_machine_learning.algorithms import VQC
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal, ZZFeatureMap, ZFeatureMap

from qiskit.circuit import ParameterVector, Parameter, QuantumCircuit
from utils import plot_decision_boundary, statevectors_for_circuit, bloch_sphere_statevector_figure

In [ ]:
def build_data_encoding_circuit(data_dim=2):
    n_qubits = 1

    data_params = ParameterVector('x', data_dim)

    model_circuit = QuantumCircuit(n_qubits)
    model_circuit.rx(data_params[0], qubit=0)
    model_circuit.ry(data_params[1], qubit=0)
    
    return model_circuit

build_data_encoding_circuit().draw('mpl')

In [ ]:
statevectors = statevectors_for_circuit(build_data_encoding_circuit(), X_train)
fig = bloch_sphere_statevector_figure(statevectors , y_train)
fig.show()

In [ ]:
def build_param_transform_circuit(n_params=3):
    n_qubits = 1

    params = ParameterVector('theta', n_params)

    model_circuit = QuantumCircuit(n_qubits)
    model_circuit.rz(params[0], qubit=0)
    model_circuit.rx(params[1], qubit=0)
    model_circuit.rz(params[2], qubit=0)

    return model_circuit

build_param_transform_circuit().draw('mpl')

In [ ]:
optimizer = SPSA(maxiter=100)
vqc = VQC(num_qubits=1,
        feature_map=build_data_encoding_circuit(),
        ansatz = build_param_transform_circuit(),
        loss = 'cross_entropy',
        optimizer = optimizer)

In [ ]:
vqc.fit(X_train, y_train)

In [ ]:
test_accuracy = vqc.score(X_test, y_test)
print(f'Testing success ratio: {test_accuracy}')

## Quantum Support Vector Machine

In [ ]:
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from utils import plot_kernel_matrix

In [ ]:
# Initialize the quantum kernel and QSVM
data_encoding_circuit = build_data_encoding_circuit()
kernel = FidelityQuantumKernel(feature_map=data_encoding_circuit)
qsvc = QSVC(quantum_kernel=kernel)

In [ ]:
plot_kernel_matrix(X_train, y_train, data_encoding_circuit)

In [ ]:
# fit the model
qsvc.fit(X_train, y_train)

In [ ]:
test_accuracy = qsvc.score(X_test, y_test)
print(f'Testing success ratio: {test_accuracy}')

## Now, it is your turn!

Try VQC and QSVM on the real-world dataset iris!